In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

In [ ]:
class grid_params:
    def __init__(self) -> None:
        
    

In [2]:
qty=np.array([-1,1]*2)*1000000
texp=[1,1,2,2]
k=[.9,1.2]*2
k_dlr=k
cp=['p','c']*2
sigma=[.20,.2]*2